In [1]:
import pandas as pd
import numpy as np

In [34]:
class Simulator():
    """
    Class to generate fake data
    """

    def __init__(self, num_rows: int):
        """
        Initialize an empty Pandas DataFrame
        """
        self.num_rows = num_rows
    
    # def get_num_rows(self):
    #     return self.num_rows
    
    def generate_categorical_column(self, num_categories: int):
        """
        Returns a Pandas Series with num_rows
        """
        return pd.Series(index = range(self.num_rows), dtype= 'object')
    
    def categorical_distribution(self, name: str = 'color', num_categories: int, distribution = 'uniform'):
        """
        Specify a categorical feature's distribution
        By default the distribution is uniform - all categories will have an equal chance of being drawn
        User can also pass a size [num_categories] array of probabilities for each category
        """
        if distribution == 'uniform':





In [35]:
a = Simulator(num_rows = 100)

In [36]:
a.generate_categorical_column(0)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
95    NaN
96    NaN
97    NaN
98    NaN
99    NaN
Length: 100, dtype: object